In [ ]:
import numpy as np
from numpy.random import randint, uniform, choice, random

In [ ]:
def generate_population(size: int, capacity: list):
    population = []
    for _ in range(size):
        population.append([uniform(1, x*0.95) for x in capacity])
    return np.array(population)

In [ ]:
def fitness_inga(capacity: np.array, flow: np.array):

    if (flow > (capacity * 0.95)).any():
        return 0

    sigma = flow.sum()
    temp = (flow/(capacity-flow)).sum()
    T = temp/sigma
    return 1/T

def gautil(flow: np.array, capacity: np.array):
    return flow/capacity


In [ ]:
def tournament_selection(size: int, pop: np.array, fitness: np.array, k=5):

    index = []
    population = list(pop)
    select = []
    for _ in range(size):
        selection_ix = randint(len(population))
        for ix in np.random.randint(0, len(population), k-1):
            if fitness[ix] > fitness[selection_ix]:
                selection_ix = ix
        select.append(population.pop(selection_ix))
        index.append(selection_ix)

    return np.array(select), np.array(index)

In [ ]:
def scatter_crossover(parent1: np.array, parent2: np.array, alpha: int, indpb = 0.8):
    if random() > indpb:
        return False

    c1 = np.copy(parent1)
    c2 = np.copy(parent2)

    n_genes = len(parent1)
    scatter_point = np.random.choice(n_genes, size=int(alpha * n_genes), replace=False)

    for i in scatter_point:
        c1[i] = parent2[i]
        c2[i] = parent1[i]

    return c1, c2

In [1]:
from deap.tools import mutGaussian

def innerga(size=10, generasi=100, capacity=None, alpha=0.5, pMu=0.2, pCr=0.8):
    pop = generate_population(size, capacity)

    for gen in range(generasi):
        fitness = np.array([fitness_inga(capacity=capacity, flow=ind) for ind in pop])

        selected, _ = tournament_selection(size=size,pop=pop, fitness=fitness)

        sh = selected[0].shape[0]
        offspring = np.empty((0, sh))

        for p1, p2 in zip(selected[::2], selected[1::2]):
            if cross := scatter_crossover(alpha=alpha, parent1=p1, parent2=p2, indpb=pCr):
                c1, c2 = cross
                offspring = np.append(offspring, c1.reshape(1, sh), axis=0)
                offspring = np.append(offspring, c2.reshape(1, sh), axis=0)
            else:
                offspring = np.append(offspring, p1.reshape(1, sh), axis=0)
                offspring = np.append(offspring, p2.reshape(1, sh), axis=0)

        for m in offspring:
            mut = mutGaussian(m, 0, 1, pMu)[0]
            offspring = np.append(offspring, mut.reshape(1, sh), axis=0)

        pop = offspring

    return pop, np.array([fitness_inga(capacity=capacity, flow=ind) for ind in pop])
